In [5]:
import pandas as pd
from datetime import datetime, date
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

ModuleNotFoundError: No module named 'sklearn'

# Load your preprocessed data

In [2]:
X = pd.read_csv('../data/model_input.csv')

# Split your dataset into training (70%) and testing (30%)

In [3]:
# try lr

In [4]:
# try rf

In [ ]:
# compare